In [1]:
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format

from IPython.display import HTML
import numpy as np
import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

## Run the below cell if you want to hide all the code cells

In [165]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Summary of what I did:
    
1. Load the data

2. Cleaning and preparing the Data:
    - get ride of trips with total amount <=0.0 and trip_distance <=0.0
    - Keep trips with only  valid values in extra of [0.00,0.50,1.00, 4.50] Assuming the 4.50 is the overnight charge
    - Keep trips with only  valid values in mta_tax of [0.00 and 0.50]
    - get ride of the trips with more than $1000 in total amount. unrealistic
    - Create trip_duration var
    - get ride of trips with durations above 5h 
    - get ride of trips with trip_distance > 0 and trip_duration = 0
    - Loading the Zone Dict: Replacing the __LocationID by the Zone Name
    - get the day of the pickup
    - get the hour of the pickup
    - get the date+day of the pickup

3. Now let explore the dataset
## let see the data in 1-D : through trip date

### Observation #1
Thursday and Friday are the most busy
Monday and Sunday are the least busy

### Observation #2
Wednesday and Thursday have the most amount and tips per trip
Saturday have the least amount and tip per trip (maybe more taxi on road? more Competition) or less people?

## Now let see the data in 2-D: day of the week and hour of the day

### Observation #3
busy hours are from 5pm to 10pm
2nd busy 8am-9am: For weekdays only
3rd busy 12pm-3pm
the effect of weekend (Saturday) staring from 9pm it become busier, peaked around midnight and then gradually decreased until 5am
on weekdays people are on streets early peaked around 8am
on weekends people are late on street peaked around 12pm

### Observation #4
most rewarding (amount per trip and tip per trip) hours
 4am-5am for weekdays
 5am-6am for weekends
2nd best 1pm-4pm for all days
3rd best 
 10pm - 1am workdays
 10pm -11pm weekends


## Now let see the data by zones

### Observation #5
Using the number of trips in each zone, Most busy area are: LaGuardia Airport, JFK Airport, Midtown, Times Sq/Theatre District, Union Sq, Upper East Side.

Since some zones may have few large amount trip, using only the number of trips to compute the most busy zones may not be very accurate. Therefore we follow the below approach:
- create a new var for each zone by dividing the trips number for each zone by the total nuumber of trips in all zones. This var will be used as a weight for each zone
- then multiple this weight by the average amount per trip and avergae tips per trip Separately. 
- finally sort


### Observation #5
based on the average trip amount per trip and average tips per trip, Most rewarded area are: LaGuardia Airport, JFK Airport, Midtown, Times Sq/Theatre District,

## let see the zone data in 2-D : zone and trip duration


### Observations for Midtown center
Busy days and hours:
Thursday and Friday
11pm-3pm | 5pm-7pm | 9pm-10pm

Most rewarding hours:
workdays: 3am-5am | 5pm - 7pm
weekends: 5am-7am

### Observations for Upper East Side South:
Busy days and hours :
Tuesday, Thursday and Friday
2pm-3pm | 5pm-7pm |
8am-9am only for workdays

Most rewarding hours:
Most rewarding days Mondays, Wednesday
workdays: 3am-5am
weekends: 5am-6am

### Observations for LaGuardia Airport and assuming JFK is the same
Busy days and hours:
Workdays (Thursday, Friday and Monday)
6pm-10pm for workdays and Sundays
9am-11am only for workdays

Most rewarding hours:
workdays: 7am-5pm
weekends: 11am-5pm

### Final Observation
Comparing taking rides from an airport zone (JFK and LaGuardia) with Manhattan
airports ride are more rewarding (tips and amounts) but take longer time to finish


## Based on the above observation, my proposed plan as a taxi driver working 10 hours a week:

on weekdays (Monday, Thursday, and Friday): work in Manhattan (Midtown center + Upper East Side South) from 3am-5am
then head to the airport area (ex. LaGuardia) and work from 1pm-3pm

on weekend (Sunday): work in Upper East Side South from 5am-6am and in Upper East Side South from 6am-7am
and if i want more hours, I will head to the LaGuardia airport after 7pm to work couple hours

## The Reasoning behind this plan:
- avoiding the busiest hours (more taxi on the street) and focusing on the hours where there are less taxi and therefore people are willing to tips more while also driving in less Crowded streets

- following this plan will result in covering both the Manhattan and airports zones in the most rewarding hours (more tips and total amount). Also, by diversifying working in both zone, I will take the advandage of the more rewarding but yet longest dutation trip from the airports


## Assumptions:
- JFK airport data follow the same trends as LaGuardia airport data
- Since we only used June 2017 data, we are assuming other months following the same trends
- Since the total amount of each trip will be calucalte based on a system (duration and length of a trip), we used the tips amount as a measurement of a zone riders' generosity. Which we consider as an extra profit of a trip.

## Possible ways to enrich this dataset
- add whether data to see if rain/snow/temp have an effect
- add the excat location of the pick up (longitude and latitude) and then applying some clustering algorithm to see if you could find clusters of more details areas than the PULOCATION zones. Then, we following the same approach to identified a more exact location to work in.
- Identify the days of holiday/celebrations or sport/social events and thier locations in NYC and see if there they affects the amount/tips of trips


## Load and Exploring the data

In [57]:

yellow_tripdata_df = pd.read_csv('/Users/zaid/Downloads/yellow_tripdata_2017-06.csv')

In [58]:
yellow_tripdata_df.shape

(9656993, 17)

## Preparing the Data

### Show me any trips with negative values in distance and amount

In [93]:
yellow_tripdata_df.loc[(yellow_tripdata_df['trip_distance']<=0.0) &(yellow_tripdata_df['total_amount']<=0.0)]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
8550,1,2017-06-09 18:51:10,2017-06-09 18:51:10,1,0.00,5,N,164,264,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10551,2,2017-06-29 16:29:01,2017-06-29 16:29:05,2,0.00,1,N,262,262,4,-2.50,-1.00,-0.50,0.00,0.00,-0.30,-4.30
26780,1,2017-06-09 20:39:01,2017-06-09 20:39:01,1,0.00,5,N,249,264,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00
33426,2,2017-06-29 16:38:08,2017-06-30 13:57:53,1,0.00,2,N,132,132,4,-52.00,-4.50,-0.50,0.00,-5.76,-0.30,-63.06
63543,2,2017-06-29 16:47:06,2017-06-29 16:47:16,5,0.00,2,N,164,164,3,-52.00,-4.50,-0.50,0.00,0.00,-0.30,-57.30
70077,1,2017-06-10 01:31:46,2017-06-10 01:31:46,1,0.00,5,N,107,264,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00
76120,1,2017-06-10 02:31:10,2017-06-10 02:31:10,1,0.00,5,Y,141,264,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00
78941,1,2017-06-10 02:59:34,2017-06-10 02:59:34,1,0.00,5,N,48,264,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00
83280,2,2017-06-29 16:52:00,2017-06-29 16:52:06,1,0.00,1,N,264,132,4,-2.50,-1.00,-0.50,0.00,0.00,-0.30,-4.30
85102,1,2017-06-10 04:18:46,2017-06-10 04:18:46,1,0.00,5,N,256,264,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Action 

## get ride of the above : total amount <=0.0 and trip_distance <=0.0

In [60]:
sub_df = yellow_tripdata_df.loc[(yellow_tripdata_df['trip_distance']>0.0) & (yellow_tripdata_df['total_amount']>0.0)]

In [61]:
sub_df.shape

(9588529, 17)

## extra field should be one of [0.00,0.50,1.00, 4.50]

In [66]:
sub_df['extra'].value_counts().sort_index()

-50.56          1
-31.61          1
 -0.49          1
 -0.45          1
 -0.35          1
  0.00    4911749
  0.02         16
  0.30        169
  0.50    3074509
  0.60          1
  0.80         63
  1.00    1561905
  1.01          1
  1.30         98
  1.50         12
  2.00          2
  2.50          1
  4.50      39987
  4.54          5
  4.80          6
Name: extra, dtype: int64

## Action
## Keep only the valid values of [0.00,0.50,1.00, 4.50] Assuming the 4.50 is the overnight charge

In [67]:
sub_df = sub_df[sub_df['extra'].isin([0.00,0.50,1.00,4.50])]
sub_df.shape

(9588150, 17)

## mta_tax field should be one of [0.00,0.50]

In [69]:
sub_df['mta_tax'].value_counts().sort_index()

 0.00      32386
 0.50    9555112
 0.85        651
53.80          1
Name: mta_tax, dtype: int64

## Action
## Keep only the valid values of [0.00 and 0.50]

In [70]:
sub_df = sub_df[sub_df['mta_tax'].isin([0.00,0.50])]
sub_df.shape

(9587498, 17)

## Show me any trips with total_amount >= $1000

In [74]:
sub_df[sub_df['total_amount']>=1000.00]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
4201741,2,2017-06-16 11:47:32,2017-06-19 08:58:12,1,"4,942.40",1,N,193,193,2,"9,999.99",0.00,0.00,0.00,0.00,0.00,"9,999.99"
5203847,2,2017-06-19 09:02:27,2017-06-20 15:20:43,1,"6,545.78",1,N,193,193,2,"9,999.99",0.00,0.00,0.00,0.00,0.00,"9,999.99"
5640037,2,2017-06-20 15:21:21,2017-06-22 11:19:32,1,"9,496.98",1,N,193,193,2,"9,999.99",0.00,0.00,0.00,0.00,0.00,"9,999.99"
5773420,1,2017-06-20 22:34:58,2017-06-20 22:39:16,1,0.80,1,N,141,141,2,"630,461.82",0.50,0.50,0.00,0.00,0.30,"630,463.12"
9218988,2,2017-06-09 16:01:22,2017-06-10 09:48:14,1,"4,208.94",1,N,193,193,2,"9,999.99",0.00,0.00,0.00,0.00,0.00,"9,999.99"


## Action
## get ride of the above records | unrealistic

In [75]:
sub_df= sub_df[sub_df['total_amount']<1000.00]
sub_df.shape

(9587493, 17)

## Create trip duration

In [76]:
sub_df['tpep_pickup_datetime'] = pd.to_datetime(sub_df['tpep_pickup_datetime'])

sub_df['tpep_dropoff_datetime'] = pd.to_datetime(sub_df['tpep_dropoff_datetime'])

In [77]:
def get_trip_duration(df):
    return df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

sub_df['trip_duration'] = sub_df['tpep_dropoff_datetime'] - sub_df['tpep_pickup_datetime']
sub_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration
0,2,2017-06-08 07:52:31,2017-06-08 08:01:32,6,1.03,1,N,161,140,1,7.50,1.00,0.50,1.86,0.00,0.30,11.16,00:09:01
1,2,2017-06-08 08:08:18,2017-06-08 08:14:00,6,1.03,1,N,162,233,1,6.00,1.00,0.50,2.34,0.00,0.30,10.14,00:05:42
2,2,2017-06-08 08:16:49,2017-06-08 15:43:22,6,5.63,1,N,137,41,2,21.50,1.00,0.50,0.00,0.00,0.30,23.30,07:26:33
3,2,2017-06-29 15:52:35,2017-06-29 16:03:27,6,1.43,1,N,142,48,1,8.50,1.00,0.50,0.88,0.00,0.30,11.18,00:10:52
4,1,2017-06-01 00:00:00,2017-06-01 00:03:43,1,0.60,1,N,140,141,1,4.50,0.50,0.50,2.00,0.00,0.30,7.80,00:03:43


In [78]:
sub_df['trip_duration'].min()

Timedelta('0 days 00:00:00')

In [79]:
sub_df['trip_duration'].max()

Timedelta('10 days 00:07:39')

## How many trips with duration above 5 hours

In [85]:
sub_df[(sub_df['trip_duration'].dt.days * 24 + sub_df['trip_duration'].dt.seconds/3600)>5].shape

(14804, 18)

## Their mean

In [86]:
sub_df[(sub_df['trip_duration'].dt.days * 24 + sub_df['trip_duration'].dt.seconds/3600)>5]['total_amount'].mean()

20.196617130502183

## get ride of trips with durations above 5h |

In [88]:
sub_df = sub_df[(sub_df['trip_duration'].dt.days * 24 + sub_df['trip_duration'].dt.seconds/3600)<5]
sub_df.shape

(9572689, 18)

## Show me trips with trip_distance > 0 and trip duration = 0 

In [98]:
sub_df.loc[(sub_df['trip_distance']>0.0) & ((sub_df['trip_duration'].dt.days * 24 * 60 * 60 + sub_df['trip_duration'].dt.seconds) == 0)]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration
4,2,2017-06-01 00:00:00,2017-06-01 00:00:00,2,17.57,2,N,132,74,1,52.00,0.00,0.50,11.71,5.76,0.30,70.27,0 days
5,2,2017-06-01 00:00:00,2017-06-01 00:00:00,5,13.34,1,N,138,249,1,47.00,0.00,0.50,10.71,5.76,0.30,64.27,0 days
6,2,2017-06-01 00:00:00,2017-06-01 00:00:00,1,18.08,2,N,132,230,2,52.00,4.50,0.50,0.00,5.76,0.30,63.06,0 days
245023,2,2017-06-01 00:12:22,2017-06-01 00:12:22,1,11.20,1,N,138,230,1,34.00,0.50,0.50,8.21,5.76,0.30,49.27,0 days
265467,2,2017-06-01 05:33:32,2017-06-01 05:33:32,1,0.18,1,N,50,246,2,2.50,0.50,0.50,0.00,0.00,0.30,3.80,0 days
417110,2,2017-06-01 15:12:09,2017-06-01 15:12:09,1,10.83,1,N,138,233,1,40.00,0.00,0.50,11.64,5.76,0.30,58.20,0 days
424077,2,2017-06-01 15:35:02,2017-06-01 15:35:02,1,0.06,1,N,28,28,2,2.50,0.00,0.50,0.00,0.00,0.30,3.30,0 days
440567,2,2017-06-01 16:36:10,2017-06-01 16:36:10,1,12.21,1,N,138,170,2,43.50,0.00,0.50,0.00,5.76,0.30,50.06,0 days
591995,2,2017-06-01 23:16:41,2017-06-01 23:16:41,1,0.05,2,N,142,142,1,52.00,0.00,0.50,10.56,0.00,0.30,63.36,0 days
604132,2,2017-06-01 23:56:57,2017-06-01 23:56:57,1,2.63,1,N,114,163,1,10.00,0.50,0.50,2.26,0.00,0.30,13.56,0 days


## get ride of trips with trip_distance > 0 and trip duration = 0 

In [102]:
sub_df = sub_df.loc[~((sub_df['trip_distance']>0.0) & ((sub_df['trip_duration'].dt.days * 24 * 60 * 60 + sub_df['trip_duration'].dt.seconds) == 0))]
sub_df.shape

(9572386, 18)

In [103]:
sub_df = sub_df.reset_index(drop=True)

## Loading the Zone Dict

In [105]:
zone_lookup_df = pd.read_csv('/Users/zaid/Downloads/taxi+_zone_lookup.csv')

In [106]:
zone_lookup_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [107]:
zone_lookup_df = zone_lookup_df.set_index('LocationID')

In [108]:
zone_lookup_dict = zone_lookup_df['Zone'].to_dict()

## Replacing the __LocationID by the Zone Name

In [110]:
sub_df['PULocationZone'] = sub_df['PULocationID'].map(zone_lookup_dict)

In [111]:
sub_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration,PULocationZone
0,2,2017-06-08 07:52:31,2017-06-08 08:01:32,6,1.03,1,N,161,140,1,7.50,1.00,0.50,1.86,0.00,0.30,11.16,00:09:01,Midtown Center
1,2,2017-06-08 08:08:18,2017-06-08 08:14:00,6,1.03,1,N,162,233,1,6.00,1.00,0.50,2.34,0.00,0.30,10.14,00:05:42,Midtown East
2,2,2017-06-29 15:52:35,2017-06-29 16:03:27,6,1.43,1,N,142,48,1,8.50,1.00,0.50,0.88,0.00,0.30,11.18,00:10:52,Lincoln Square East
3,1,2017-06-01 00:00:00,2017-06-01 00:03:43,1,0.60,1,N,140,141,1,4.50,0.50,0.50,2.00,0.00,0.30,7.80,00:03:43,Lenox Hill East
4,1,2017-06-01 00:00:01,2017-06-01 00:01:43,1,0.20,1,N,148,148,2,3.00,0.50,0.50,0.00,0.00,0.30,4.30,00:01:42,Lower East Side


## get the day of the pickup

In [112]:
sub_df['tpep_pickup_datetime_day'] = sub_df['tpep_pickup_datetime'].apply(lambda x: x.day_name())

## get the hour of the pickup

In [113]:
sub_df['tpep_pickup_datetime_hour'] = sub_df['tpep_pickup_datetime'].apply(lambda x: x.hour)

## get the date+day of the pickup

In [114]:
sub_df['tpep_pickup_datetime_date_day'] = sub_df['tpep_pickup_datetime'].apply(lambda x: str(x.date())+'_'+x.day_name())

## Now let explore the dataset
## First on 1-D : through trip date

In [115]:
summary_day_rides = pd.DataFrame(sub_df.groupby(['tpep_pickup_datetime_date_day']).size())
summary_day_rides = summary_day_rides.rename(columns = {0:'Number of Trips'})
summary_day_rides.reset_index(level=0, inplace=True)

data = go.Scatter(
                x=summary_day_rides['tpep_pickup_datetime_date_day'],
                y=summary_day_rides['Number of Trips'],
               
                #name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

layout = dict(
    title = "Number of trips daily",

)

fig = dict(data=[data], layout=layout)
plotly.offline.iplot(fig, filename = "Number of trips daily")

# Observation #1
## Thursday and Friday are the most busy 
## Monday and Sunday are the less busy

In [116]:
def get_average_taxi(df):
    return df['total_amount'].sum()/float(len(df))

summary_day_average_taxi = pd.DataFrame(sub_df.groupby(['tpep_pickup_datetime_date_day']).apply(get_average_taxi))


summary_day_average_taxi = summary_day_average_taxi.rename(columns = {0:'average_taxi'})
summary_day_average_taxi.reset_index(level=0, inplace=True)


data = go.Scatter(
                x=summary_day_average_taxi['tpep_pickup_datetime_date_day'],
                y=summary_day_average_taxi['average_taxi'],
                line = dict(color = '#17BECF'),
                opacity = 0.8)



layout = dict(
    title = "Amount per trips daily",

)

fig = dict(data=[data], layout=layout)
plotly.offline.iplot(fig, filename = "Amount per trips daily")

In [117]:
def get_average_tip(df):
    return df['tip_amount'].sum()/float(len(df))

summary_day_average_tip = pd.DataFrame(sub_df.groupby(['tpep_pickup_datetime_date_day']).apply(get_average_tip))


summary_day_average_tip = summary_day_average_tip.rename(columns = {0:'average_tip'})
summary_day_average_tip.reset_index(level=0, inplace=True)


data = go.Scatter(
                x=summary_day_average_tip['tpep_pickup_datetime_date_day'],
                y=summary_day_average_tip['average_tip'],
                line = dict(color = '#17BECF'),
                opacity = 0.8)



layout = dict(
    title = "Tip per trips daily",

)

fig = dict(data=[data], layout=layout)
plotly.offline.iplot(fig, filename = "Tip per trips daily")

# Observation #2
## Wednesday and Thursday have the most amount and tip per trip 
## Saturday have the less amount and tip per trip (maybe more taxi on road? more Competition) or less people?

## Now let see the data in 2-D: day of the week and hour of the day

In [149]:
summary_day_hour_rides = pd.DataFrame(sub_df.groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).size())
summary_day_hour_rides = summary_day_hour_rides.rename(columns = {0:'Number of Trips'})
summary_day_hour_rides.reset_index(level=0, inplace=True)


Monday = go.Scatter(
                x=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Monday']['Number of Trips'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Tuesday']['Number of Trips'],
                name = "Tuesday",

                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Wednesday']['Number of Trips'],
                name = "Wednesday",

                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Thursday']['Number of Trips'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Friday']['Number of Trips'],
                name = "Friday",

                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Saturday']['Number of Trips'],
                name = "Saturday",

                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=summary_day_hour_rides[summary_day_hour_rides['tpep_pickup_datetime_day'] == 'Sunday']['Number of Trips'],
                name = "Sunday",
    
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Number of trips hourly per day",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Number of trips hourly per day")

# Observation #3

## busy hours are from 5pm to 10pm
## 2nd busy 8am-9am:  For weekdays only

## 3rd busy 12pm-3pm

## the effect of weekend (Saturday) staring from 9pm it become busier, peaked around midnight and then gradually decreased until 5am

## on weekdays people are on streets early peaked around 8am
## on weekends people are late on street peaked around 12pm

In [150]:
summary_day_hour_amount = pd.DataFrame(sub_df.groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour'])['total_amount'].sum())
#summary_day_hour_rides = summary_day_hour_rides.rename(columns = {0:'Number of Trips'})
summary_day_hour_amount.reset_index(level=0, inplace=True)



Monday = go.Scatter(
                x=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Monday']['total_amount'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Tuesday']['total_amount'],
                name = "Tuesday",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Wednesday']['total_amount'],
                name = "Wednesday",
                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Thursday']['total_amount'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Friday']['total_amount'],
                name = "Friday",
                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Saturday']['total_amount'],
                name = "Saturday",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=summary_day_hour_amount[summary_day_hour_amount['tpep_pickup_datetime_day'] == 'Sunday']['total_amount'],
                name = "Sunday",
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Amount of trips hourly per day",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Amount of trips hourly per day")

In [151]:
def get_average_taxi_by_hour(df):
    return df['total_amount'].sum()/float(len(df))

summary_day_hour_average_taxi_by_hour = pd.DataFrame(sub_df.groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).apply(get_average_taxi_by_hour))


summary_day_hour_average_taxi_by_hour = summary_day_hour_average_taxi_by_hour.rename(columns = {0:'average_taxi_by_hour'})
summary_day_hour_average_taxi_by_hour.reset_index(level=0, inplace=True)





Monday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Monday']['average_taxi_by_hour'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Tuesday']['average_taxi_by_hour'],
                name = "Tuesday",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Wednesday']['average_taxi_by_hour'],
                name = "Wednesday",
                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Thursday']['average_taxi_by_hour'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Friday']['average_taxi_by_hour'],
                name = "Friday",
                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Saturday']['average_taxi_by_hour'],
                name = "Saturday",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Sunday']['average_taxi_by_hour'],
                name = "Sunday",
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Amount per trips hourly per day",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Amount per trips hourly per day")

In [152]:
summary_day_hour_tip = pd.DataFrame(sub_df.groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour'])['tip_amount'].sum())
#summary_day_hour_rides = summary_day_hour_rides.rename(columns = {0:'Number of Trips'})
summary_day_hour_tip.reset_index(level=0, inplace=True)




Monday = go.Scatter(
                x=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Monday']['tip_amount'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Tuesday']['tip_amount'],
                name = "Tuesday",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Wednesday']['tip_amount'],
                name = "Wednesday",
                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Thursday']['tip_amount'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Friday']['tip_amount'],
                name = "Friday",
                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Saturday']['tip_amount'],
                name = "Saturday",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=summary_day_hour_tip[summary_day_hour_tip['tpep_pickup_datetime_day'] == 'Sunday']['tip_amount'],
                name = "Sunday",
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Tip trips hourly per day",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Tip trips hourly per day")

In [153]:
def get_average_tip_by_hour(df):
    return df['tip_amount'].sum()/float(len(df))

summary_day_hour_average_tip_by_hour = pd.DataFrame(sub_df.groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).apply(get_average_tip_by_hour))


summary_day_hour_average_tip_by_hour = summary_day_hour_average_tip_by_hour.rename(columns = {0:'average_tip_by_hour'})
summary_day_hour_average_tip_by_hour.reset_index(level=0, inplace=True)




Monday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Monday']['average_tip_by_hour'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Tuesday']['average_tip_by_hour'],
                name = "Tuesday",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Wednesday']['average_tip_by_hour'],
                name = "Wednesday",
                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Thursday']['average_tip_by_hour'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Friday']['average_tip_by_hour'],
                name = "Friday",
                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Saturday']['average_tip_by_hour'],
                name = "Saturday",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Sunday']['average_tip_by_hour'],
                name = "Sunday",
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Tip per trips hourly per day",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Tip per trips hourly per day")

# Observation #4
## most rewarding (amount per trip and tip per trip) hours  
    ## 4am-5am for weekdays
    ## 5am-6am for weekends
## 2nd rewarding 1pm-4pm for all days
## 3rd  rewarding
    ## 10pm - 1am workdays
    ## 10pm -11pm weekends


## Now let see the data by zones

In [123]:
sub_df.groupby('PULocationZone')['total_amount'].sum().sort_values(ascending = False)

PULocationZone
JFK Airport                                            12,769,679.81
LaGuardia Airport                                      11,878,029.03
Midtown Center                                          5,383,762.71
Times Sq/Theatre District                               5,149,153.05
Midtown East                                            4,941,588.25
Penn Station/Madison Sq West                            4,913,971.48
Murray Hill                                             4,585,503.53
Upper East Side South                                   4,565,737.82
Clinton East                                            4,340,306.68
Upper East Side North                                   4,309,901.73
Union Sq                                                4,299,998.51
Midtown North                                           3,937,460.59
East Village                                            3,903,987.40
Lincoln Square East                                     3,725,496.94
Midtown South      

In [124]:
sub_df.groupby('PULocationZone').size().sort_values(ascending = False)

PULocationZone
Upper East Side South                            377935
Midtown Center                                   356853
Upper East Side North                            341831
Midtown East                                     329724
Penn Station/Madison Sq West                     326630
Murray Hill                                      313396
Times Sq/Theatre District                        307580
Union Sq                                         306454
Clinton East                                     300053
Lincoln Square East                              274629
East Village                                     272363
LaGuardia Airport                                266330
Midtown North                                    256110
Upper West Side South                            244863
Gramercy                                         226315
Midtown South                                    226168
JFK Airport                                      225713
East Chelsea                     

# Observation #5
## Most busy area are: LaGuardia Airport, JFK Airport, Midtown, Times Sq/Theatre District, Union Sq, Upper East Side

## Create new summary vars for each zone

In [125]:
area_zone_df = pd.DataFrame (sub_df.groupby('PULocationZone').size())
area_zone_df = area_zone_df.rename(columns = {0:'Number_of_trips'})

area_zone_df.head()

,Number_of_trips
PULocationZone,
Allerton/Pelham Gardens,22
Alphabet City,23267
Arrochar/Fort Wadsworth,35
Astoria,16682
Astoria Park,135


In [141]:
sub_df['trip_duration_minutes'] = sub_df['trip_duration'].dt.days * 24 * 60  + sub_df['trip_duration'].dt.seconds/60
sub_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration,PULocationZone,tpep_pickup_datetime_day,tpep_pickup_datetime_hour,tpep_pickup_datetime_date_day,trip_duration_minutes
0,2,2017-06-08 07:52:31,2017-06-08 08:01:32,6,1.03,1,N,161,140,1,...,1.86,0.00,0.30,11.16,00:09:01,Midtown Center,Thursday,7,2017-06-08_Thursday,9.02
1,2,2017-06-08 08:08:18,2017-06-08 08:14:00,6,1.03,1,N,162,233,1,...,2.34,0.00,0.30,10.14,00:05:42,Midtown East,Thursday,8,2017-06-08_Thursday,5.70
2,2,2017-06-29 15:52:35,2017-06-29 16:03:27,6,1.43,1,N,142,48,1,...,0.88,0.00,0.30,11.18,00:10:52,Lincoln Square East,Thursday,15,2017-06-29_Thursday,10.87
3,1,2017-06-01 00:00:00,2017-06-01 00:03:43,1,0.60,1,N,140,141,1,...,2.00,0.00,0.30,7.80,00:03:43,Lenox Hill East,Thursday,0,2017-06-01_Thursday,3.72
4,1,2017-06-01 00:00:01,2017-06-01 00:01:43,1,0.20,1,N,148,148,2,...,0.00,0.00,0.30,4.30,00:01:42,Lower East Side,Thursday,0,2017-06-01_Thursday,1.70


In [144]:
area_zone_df0 = pd.DataFrame (sub_df.groupby('PULocationZone')['trip_duration_minutes'].mean())

area_zone_df = pd.merge(area_zone_df, area_zone_df0, how='left', on=['PULocationZone'])
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight,average_taxi_amount_X_Number_of_trips_weight,trip_duration_minutes
PULocationZone,,,,,,,,,
Allerton/Pelham Gardens,22,480.54,21.84,87.32,3.97,0.00,0.00,0.01,14.81
Alphabet City,23267,"343,979.90",14.78,"36,613.07",1.57,0.24,0.38,3.59,13.27
Arrochar/Fort Wadsworth,35,"1,244.90",35.57,94.65,2.70,0.00,0.00,0.01,11.14
Astoria,16682,"228,180.87",13.68,"17,364.66",1.04,0.17,0.18,2.38,11.68
Astoria Park,135,"3,645.38",27.00,416.94,3.09,0.00,0.00,0.04,16.36


In [126]:
area_zone_df2 = pd.DataFrame (sub_df.groupby('PULocationZone')['total_amount'].sum())

area_zone_df = pd.merge(area_zone_df, area_zone_df2, how='left', on=['PULocationZone'])
area_zone_df.head()

,Number_of_trips,total_amount
PULocationZone,,
Allerton/Pelham Gardens,22,480.54
Alphabet City,23267,"343,979.90"
Arrochar/Fort Wadsworth,35,"1,244.90"
Astoria,16682,"228,180.87"
Astoria Park,135,"3,645.38"


In [127]:
area_zone_df['average_taxi_amount'] = area_zone_df['total_amount'] / area_zone_df['Number_of_trips']
area_zone_df.head()



,Number_of_trips,total_amount,average_taxi_amount
PULocationZone,,,
Allerton/Pelham Gardens,22,480.54,21.84
Alphabet City,23267,"343,979.90",14.78
Arrochar/Fort Wadsworth,35,"1,244.90",35.57
Astoria,16682,"228,180.87",13.68
Astoria Park,135,"3,645.38",27.00


In [128]:
area_zone_df3 = pd.DataFrame (sub_df.groupby('PULocationZone')['tip_amount'].sum())

area_zone_df = pd.merge(area_zone_df, area_zone_df3, how='left', on=['PULocationZone'])
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount
PULocationZone,,,,
Allerton/Pelham Gardens,22,480.54,21.84,87.32
Alphabet City,23267,"343,979.90",14.78,"36,613.07"
Arrochar/Fort Wadsworth,35,"1,244.90",35.57,94.65
Astoria,16682,"228,180.87",13.68,"17,364.66"
Astoria Park,135,"3,645.38",27.00,416.94


In [129]:
area_zone_df['average_tip'] = area_zone_df['tip_amount'] / area_zone_df['Number_of_trips']
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip
PULocationZone,,,,,
Allerton/Pelham Gardens,22,480.54,21.84,87.32,3.97
Alphabet City,23267,"343,979.90",14.78,"36,613.07",1.57
Arrochar/Fort Wadsworth,35,"1,244.90",35.57,94.65,2.70
Astoria,16682,"228,180.87",13.68,"17,364.66",1.04
Astoria Park,135,"3,645.38",27.00,416.94,3.09


## This var (Number_of_trips_weight) take in considration the total number of trips for each zone in reference with the total number of trips in all zones

## As a result we will consider this var as weight for each zone to measure its important based on how busy it is

In [130]:
area_zone_df['Number_of_trips_weight'] = area_zone_df['Number_of_trips'] / float(area_zone_df['Number_of_trips'].sum()) * 100
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight
PULocationZone,,,,,,
Allerton/Pelham Gardens,22,480.54,21.84,87.32,3.97,0.00
Alphabet City,23267,"343,979.90",14.78,"36,613.07",1.57,0.24
Arrochar/Fort Wadsworth,35,"1,244.90",35.57,94.65,2.70,0.00
Astoria,16682,"228,180.87",13.68,"17,364.66",1.04,0.17
Astoria Park,135,"3,645.38",27.00,416.94,3.09,0.00


## Now we multiple the average tip (and then average amount) with each zone weight


## this var (average_tip_X_Number_of_trips_weight) will measure how generous an average rider from this zone

In [131]:
area_zone_df['average_tip_X_Number_of_trips_weight'] = area_zone_df['average_tip'] * area_zone_df['Number_of_trips_weight']
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight
PULocationZone,,,,,,,
Allerton/Pelham Gardens,22,480.54,21.84,87.32,3.97,0.00,0.00
Alphabet City,23267,"343,979.90",14.78,"36,613.07",1.57,0.24,0.38
Arrochar/Fort Wadsworth,35,"1,244.90",35.57,94.65,2.70,0.00,0.00
Astoria,16682,"228,180.87",13.68,"17,364.66",1.04,0.17,0.18
Astoria Park,135,"3,645.38",27.00,416.94,3.09,0.00,0.00


## this var will measure the weigted amount of an average trip starting from this zone

In [132]:
area_zone_df['average_taxi_amount_X_Number_of_trips_weight'] = area_zone_df['average_taxi_amount'] * area_zone_df['Number_of_trips_weight']
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight,average_taxi_amount_X_Number_of_trips_weight
PULocationZone,,,,,,,,
Allerton/Pelham Gardens,22,480.54,21.84,87.32,3.97,0.00,0.00,0.01
Alphabet City,23267,"343,979.90",14.78,"36,613.07",1.57,0.24,0.38,3.59
Arrochar/Fort Wadsworth,35,"1,244.90",35.57,94.65,2.70,0.00,0.00,0.01
Astoria,16682,"228,180.87",13.68,"17,364.66",1.04,0.17,0.18,2.38
Astoria Park,135,"3,645.38",27.00,416.94,3.09,0.00,0.00,0.04


## Finally we sort the zones based on the weighted average amount for each trip

In [133]:
area_zone_df.sort_values(by=['average_taxi_amount_X_Number_of_trips_weight'],ascending=False)

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight,average_taxi_amount_X_Number_of_trips_weight
PULocationZone,,,,,,,,
JFK Airport,225713,"12,769,679.81",56.57,"1,317,458.54",5.84,2.36,13.77,133.43
LaGuardia Airport,266330,"11,878,029.03",44.60,"1,559,963.86",5.86,2.78,16.30,124.12
Midtown Center,356853,"5,383,762.71",15.09,"627,710.61",1.76,3.73,6.56,56.26
Times Sq/Theatre District,307580,"5,149,153.05",16.74,"555,120.21",1.80,3.21,5.80,53.80
Midtown East,329724,"4,941,588.25",14.99,"595,031.01",1.80,3.45,6.22,51.64
Penn Station/Madison Sq West,326630,"4,913,971.48",15.04,"539,118.77",1.65,3.41,5.63,51.35
Murray Hill,313396,"4,585,503.53",14.63,"540,537.42",1.72,3.27,5.65,47.91
Upper East Side South,377935,"4,565,737.82",12.08,"502,542.18",1.33,3.95,5.25,47.71
Clinton East,300053,"4,340,306.68",14.47,"462,546.31",1.54,3.14,4.83,45.35


## and we sort the zones based on the weighted average tip for each trip

In [134]:
area_zone_df.sort_values(by=['average_tip_X_Number_of_trips_weight'],ascending=False)

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight,average_taxi_amount_X_Number_of_trips_weight
PULocationZone,,,,,,,,
LaGuardia Airport,266330,"11,878,029.03",44.60,"1,559,963.86",5.86,2.78,16.30,124.12
JFK Airport,225713,"12,769,679.81",56.57,"1,317,458.54",5.84,2.36,13.77,133.43
Midtown Center,356853,"5,383,762.71",15.09,"627,710.61",1.76,3.73,6.56,56.26
Midtown East,329724,"4,941,588.25",14.99,"595,031.01",1.80,3.45,6.22,51.64
Times Sq/Theatre District,307580,"5,149,153.05",16.74,"555,120.21",1.80,3.21,5.80,53.80
Murray Hill,313396,"4,585,503.53",14.63,"540,537.42",1.72,3.27,5.65,47.91
Penn Station/Madison Sq West,326630,"4,913,971.48",15.04,"539,118.77",1.65,3.41,5.63,51.35
Union Sq,306454,"4,299,998.51",14.03,"522,485.41",1.70,3.20,5.46,44.93
Upper East Side South,377935,"4,565,737.82",12.08,"502,542.18",1.33,3.95,5.25,47.71


# Observation #5
## Most rewarded area are: LaGuardia Airport, JFK Airport, Midtown, Times Sq/Theatre District, 

## let see the data in 2-D : zone and trip duration

## create an average trip duration (in min) for each zone

In [145]:
sub_df['trip_duration_minutes'] = sub_df['trip_duration'].dt.days * 24 * 60  + sub_df['trip_duration'].dt.seconds/60
sub_df.head()

area_zone_df0 = pd.DataFrame (sub_df.groupby('PULocationZone')['trip_duration_minutes'].mean())

area_zone_df = pd.merge(area_zone_df, area_zone_df0, how='left', on=['PULocationZone'])
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight,average_taxi_amount_X_Number_of_trips_weight,trip_duration_minutes_x,trip_duration_minutes_y
PULocationZone,,,,,,,,,,
Allerton/Pelham Gardens,22,480.54,21.84,87.32,3.97,0.00,0.00,0.01,14.81,14.81
Alphabet City,23267,"343,979.90",14.78,"36,613.07",1.57,0.24,0.38,3.59,13.27,13.27
Arrochar/Fort Wadsworth,35,"1,244.90",35.57,94.65,2.70,0.00,0.00,0.01,11.14,11.14
Astoria,16682,"228,180.87",13.68,"17,364.66",1.04,0.17,0.18,2.38,11.68,11.68
Astoria Park,135,"3,645.38",27.00,416.94,3.09,0.00,0.00,0.04,16.36,16.36


In [147]:
area_zone_df['trip_duration_minutes_X_Number_of_trips_weight'] = area_zone_df['trip_duration_minutes_x'] * area_zone_df['Number_of_trips_weight']
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight,average_taxi_amount_X_Number_of_trips_weight,trip_duration_minutes_x,trip_duration_minutes_y,trip_duration_minutes_X_Number_of_trips_weight
PULocationZone,,,,,,,,,,,
Allerton/Pelham Gardens,22,480.54,21.84,87.32,3.97,0.00,0.00,0.01,14.81,14.81,0.00
Alphabet City,23267,"343,979.90",14.78,"36,613.07",1.57,0.24,0.38,3.59,13.27,13.27,3.23
Arrochar/Fort Wadsworth,35,"1,244.90",35.57,94.65,2.70,0.00,0.00,0.01,11.14,11.14,0.00
Astoria,16682,"228,180.87",13.68,"17,364.66",1.04,0.17,0.18,2.38,11.68,11.68,2.04
Astoria Park,135,"3,645.38",27.00,416.94,3.09,0.00,0.00,0.04,16.36,16.36,0.02


In [148]:
area_zone_df.sort_values(by=['trip_duration_minutes_X_Number_of_trips_weight'],ascending=False)

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight,average_taxi_amount_X_Number_of_trips_weight,trip_duration_minutes_x,trip_duration_minutes_y,trip_duration_minutes_X_Number_of_trips_weight
PULocationZone,,,,,,,,,,,
JFK Airport,225713,"12,769,679.81",56.57,"1,317,458.54",5.84,2.36,13.77,133.43,44.97,44.97,106.06
LaGuardia Airport,266330,"11,878,029.03",44.60,"1,559,963.86",5.86,2.78,16.30,124.12,37.53,37.53,104.45
Midtown Center,356853,"5,383,762.71",15.09,"627,710.61",1.76,3.73,6.56,56.26,14.27,14.27,53.20
Penn Station/Madison Sq West,326630,"4,913,971.48",15.04,"539,118.77",1.65,3.41,5.63,51.35,15.24,15.24,52.03
Times Sq/Theatre District,307580,"5,149,153.05",16.74,"555,120.21",1.80,3.21,5.80,53.80,15.51,15.51,49.86
Midtown East,329724,"4,941,588.25",14.99,"595,031.01",1.80,3.45,6.22,51.64,13.71,13.71,47.25
Murray Hill,313396,"4,585,503.53",14.63,"540,537.42",1.72,3.27,5.65,47.91,13.45,13.45,44.04
Upper East Side South,377935,"4,565,737.82",12.08,"502,542.18",1.33,3.95,5.25,47.71,11.13,11.13,43.95
Union Sq,306454,"4,299,998.51",14.03,"522,485.41",1.70,3.20,5.46,44.93,13.47,13.47,43.12


In [154]:
df = pd.DataFrame(sub_df.groupby('PULocationZone').get_group('Midtown Center').groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).size())
df = df.rename(columns = {0:'Number of Trips'})
df.reset_index(level=0, inplace=True)

Monday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Monday']['Number of Trips'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Tuesday']['Number of Trips'],
                name = "Tuesday",

                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Wednesday']['Number of Trips'],
                name = "Wednesday",

                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Thursday']['Number of Trips'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Friday']['Number of Trips'],
                name = "Friday",

                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Saturday']['Number of Trips'],
                name = "Saturday",

                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Sunday']['Number of Trips'],
                name = "Sunday",
    
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Number of trips hourly per day for Midtown Center",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Number of trips hourly per day for Midtown Center")

# Observation

## Busy days and hours for Midtown center: 
## Thursday and Friday 
## 11pm-3pm | 5pm-7pm | 9pm-10pm

In [161]:

def get_average_taxi_by_hour(df):
    return df['total_amount'].sum()/float(len(df))

summary_day_hour_average_taxi_by_hour = pd.DataFrame(sub_df.groupby('PULocationZone').get_group('Midtown Center').groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).apply(get_average_taxi_by_hour))


summary_day_hour_average_taxi_by_hour = summary_day_hour_average_taxi_by_hour.rename(columns = {0:'average_taxi_by_hour'})
summary_day_hour_average_taxi_by_hour.reset_index(level=0, inplace=True)





Monday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Monday']['average_taxi_by_hour'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Tuesday']['average_taxi_by_hour'],
                name = "Tuesday",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Wednesday']['average_taxi_by_hour'],
                name = "Wednesday",
                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Thursday']['average_taxi_by_hour'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Friday']['average_taxi_by_hour'],
                name = "Friday",
                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Saturday']['average_taxi_by_hour'],
                name = "Saturday",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Sunday']['average_taxi_by_hour'],
                name = "Sunday",
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Amount per trips hourly per day for Midtown Center",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Amount per trips hourly per day for Midtown Center")

In [162]:

def get_average_tip_by_hour(df):
    return df['tip_amount'].sum()/float(len(df))

summary_day_hour_average_tip_by_hour = pd.DataFrame(sub_df.groupby('PULocationZone').get_group('Midtown Center').groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).apply(get_average_tip_by_hour))


summary_day_hour_average_tip_by_hour = summary_day_hour_average_tip_by_hour.rename(columns = {0:'average_tip_by_hour'})
summary_day_hour_average_tip_by_hour.reset_index(level=0, inplace=True)




Monday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Monday']['average_tip_by_hour'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Tuesday']['average_tip_by_hour'],
                name = "Tuesday",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Wednesday']['average_tip_by_hour'],
                name = "Wednesday",
                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Thursday']['average_tip_by_hour'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Friday']['average_tip_by_hour'],
                name = "Friday",
                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Saturday']['average_tip_by_hour'],
                name = "Saturday",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Sunday']['average_tip_by_hour'],
                name = "Sunday",
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Tip per trips hourly per day for Midtown Center",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Tip per trips hourly per day for Midtown Center")

## Observation 



## Most rewarding hours for Midtown Center


## workdays: 3am-5am | 5pm - 7pm
## weekends: 5am-7am

In [155]:

df = pd.DataFrame(sub_df.groupby('PULocationZone').get_group('Upper East Side South').groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).size())
df = df.rename(columns = {0:'Number of Trips'})
df.reset_index(level=0, inplace=True)

Monday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Monday']['Number of Trips'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Tuesday']['Number of Trips'],
                name = "Tuesday",

                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Wednesday']['Number of Trips'],
                name = "Wednesday",

                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Thursday']['Number of Trips'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Friday']['Number of Trips'],
                name = "Friday",

                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Saturday']['Number of Trips'],
                name = "Saturday",

                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Sunday']['Number of Trips'],
                name = "Sunday",
    
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Number of trips hourly per day for Upper East Side South",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Number of trips hourly per day for Upper East Side South")

# Observation #6

## Busy days and hours for Upper East Side South: 
## Tuesday, Thursday and Friday 
## 2pm-3pm | 5pm-7pm | 
## 8am-9am only for workdays


In [156]:
def get_average_taxi_by_hour(df):
    return df['total_amount'].sum()/float(len(df))

summary_day_hour_average_taxi_by_hour = pd.DataFrame(sub_df.groupby('PULocationZone').get_group('Upper East Side South').groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).apply(get_average_taxi_by_hour))


summary_day_hour_average_taxi_by_hour = summary_day_hour_average_taxi_by_hour.rename(columns = {0:'average_taxi_by_hour'})
summary_day_hour_average_taxi_by_hour.reset_index(level=0, inplace=True)





Monday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Monday']['average_taxi_by_hour'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Tuesday']['average_taxi_by_hour'],
                name = "Tuesday",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Wednesday']['average_taxi_by_hour'],
                name = "Wednesday",
                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Thursday']['average_taxi_by_hour'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Friday']['average_taxi_by_hour'],
                name = "Friday",
                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Saturday']['average_taxi_by_hour'],
                name = "Saturday",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Sunday']['average_taxi_by_hour'],
                name = "Sunday",
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Amount per trips hourly per day for Upper East Side South",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Amount per trips hourly per day for Upper East Side South")

In [157]:
def get_average_tip_by_hour(df):
    return df['tip_amount'].sum()/float(len(df))

summary_day_hour_average_tip_by_hour = pd.DataFrame(sub_df.groupby('PULocationZone').get_group('Upper East Side South').groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).apply(get_average_tip_by_hour))


summary_day_hour_average_tip_by_hour = summary_day_hour_average_tip_by_hour.rename(columns = {0:'average_tip_by_hour'})
summary_day_hour_average_tip_by_hour.reset_index(level=0, inplace=True)




Monday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Monday']['average_tip_by_hour'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Tuesday']['average_tip_by_hour'],
                name = "Tuesday",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Wednesday']['average_tip_by_hour'],
                name = "Wednesday",
                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Thursday']['average_tip_by_hour'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Friday']['average_tip_by_hour'],
                name = "Friday",
                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Saturday']['average_tip_by_hour'],
                name = "Saturday",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Sunday']['average_tip_by_hour'],
                name = "Sunday",
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Tip per trips hourly per day for Upper East Side South",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Tip per trips hourly per day for Upper East Side South")

## Observation #8

## Most rewarding hours for upper East Side south

## Most rewarding days Mondays, Wednesday 



## workdays: 3am-5am
## weekends: 5am-6am

In [158]:

df = pd.DataFrame(sub_df.groupby('PULocationZone').get_group('LaGuardia Airport').groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).size())
df = df.rename(columns = {0:'Number of Trips'})
df.reset_index(level=0, inplace=True)

Monday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Monday']['Number of Trips'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Tuesday']['Number of Trips'],
                name = "Tuesday",

                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Wednesday']['Number of Trips'],
                name = "Wednesday",

                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Thursday']['Number of Trips'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Friday']['Number of Trips'],
                name = "Friday",

                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Saturday']['Number of Trips'],
                name = "Saturday",

                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=df[df['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=df[df['tpep_pickup_datetime_day'] == 'Sunday']['Number of Trips'],
                name = "Sunday",
    
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Number of trips hourly per day for LaGuardia Airport",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Number of trips hourly per day for LaGuardia Airport")

# Observation #9

## Busy days and hours for LaGuardia Airport and assuming JFK is the same: 
## Workdays (Thursday, Friday and Monday)
## 6pm-10pm for workdays and Sundays

## 9am-11am only for workdays

In [159]:

def get_average_taxi_by_hour(df):
    return df['total_amount'].sum()/float(len(df))

summary_day_hour_average_taxi_by_hour = pd.DataFrame(sub_df.groupby('PULocationZone').get_group('LaGuardia Airport').groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).apply(get_average_taxi_by_hour))


summary_day_hour_average_taxi_by_hour = summary_day_hour_average_taxi_by_hour.rename(columns = {0:'average_taxi_by_hour'})
summary_day_hour_average_taxi_by_hour.reset_index(level=0, inplace=True)





Monday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Monday']['average_taxi_by_hour'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Tuesday']['average_taxi_by_hour'],
                name = "Tuesday",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Wednesday']['average_taxi_by_hour'],
                name = "Wednesday",
                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Thursday']['average_taxi_by_hour'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Friday']['average_taxi_by_hour'],
                name = "Friday",
                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Saturday']['average_taxi_by_hour'],
                name = "Saturday",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=summary_day_hour_average_taxi_by_hour[summary_day_hour_average_taxi_by_hour['tpep_pickup_datetime_day'] == 'Sunday']['average_taxi_by_hour'],
                name = "Sunday",
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Amount per trips hourly per day for LaGuardia Airport",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Amount per trips hourly per day for LaGuardia Airport")



In [160]:

def get_average_tip_by_hour(df):
    return df['tip_amount'].sum()/float(len(df))

summary_day_hour_average_tip_by_hour = pd.DataFrame(sub_df.groupby('PULocationZone').get_group('LaGuardia Airport').groupby(['tpep_pickup_datetime_day','tpep_pickup_datetime_hour']).apply(get_average_tip_by_hour))


summary_day_hour_average_tip_by_hour = summary_day_hour_average_tip_by_hour.rename(columns = {0:'average_tip_by_hour'})
summary_day_hour_average_tip_by_hour.reset_index(level=0, inplace=True)




Monday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Monday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Monday']['average_tip_by_hour'],
                name = "Monday",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

Tuesday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Tuesday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Tuesday']['average_tip_by_hour'],
                name = "Tuesday",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

Wednesday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Wednesday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Wednesday']['average_tip_by_hour'],
                name = "Wednesday",
                line = dict(color = '#FFFF00'),
                opacity = 0.8)

Thursday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Thursday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Thursday']['average_tip_by_hour'],
                name = "Thursday",
                line = dict(color = '#00FFFF'),
                opacity = 0.8)

Friday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Friday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Friday']['average_tip_by_hour'],
                name = "Friday",
                line = dict(color = '#FF00FF'),
                opacity = 0.8)

Saturday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Saturday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Saturday']['average_tip_by_hour'],
                name = "Saturday",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

Sunday = go.Scatter(
                x=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Sunday'].index,
                y=summary_day_hour_average_tip_by_hour[summary_day_hour_average_tip_by_hour['tpep_pickup_datetime_day'] == 'Sunday']['average_tip_by_hour'],
                name = "Sunday",
                line = dict(color = '#0000FF'),
                opacity = 0.8)



data = [Monday, Tuesday, Wednesday , Thursday , Friday, Saturday, Sunday]

layout = dict(
    title = "Tip per trips hourly per day for LaGuardia Airport",

)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Tip per trips hourly per day for LaGuardia Airport")

## Observation #10

## Most rewarding hours for LaGuardia Airport

## workdays: 7am-5pm
## weekends: 11am-5pm

## Observation

## Comparing taking rides from an airport zone (JFK and LaGuardia) with Manhattan
### airports ride are more rewarding (tips and amounts) but take longer time to finish

In [263]:
area_zone_df2 = pd.DataFrame (sub_df.groupby('PULocationZone')['total_amount'].sum())

area_zone_df = pd.merge(area_zone_df, area_zone_df2, how='left', on=['PULocationZone'])
area_zone_df.head()

,Number_of_trips,total_amount
PULocationZone,,
Allerton/Pelham Gardens,14,375.14
Alphabet City,14354,"233,040.60"
Arrochar/Fort Wadsworth,10,649.45
Astoria,6286,"106,404.71"
Astoria Park,74,"2,339.00"


In [264]:
area_zone_df['average_taxi_amount'] = area_zone_df['total_amount'] / area_zone_df['Number_of_trips']
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount
PULocationZone,,,
Allerton/Pelham Gardens,14,375.14,26.80
Alphabet City,14354,"233,040.60",16.24
Arrochar/Fort Wadsworth,10,649.45,64.94
Astoria,6286,"106,404.71",16.93
Astoria Park,74,"2,339.00",31.61


In [265]:

area_zone_df3 = pd.DataFrame (sub_df.groupby('PULocationZone')['tip_amount'].sum())

area_zone_df = pd.merge(area_zone_df, area_zone_df3, how='left', on=['PULocationZone'])
area_zone_df.head()


,Number_of_trips,total_amount,average_taxi_amount,tip_amount
PULocationZone,,,,
Allerton/Pelham Gardens,14,375.14,26.80,87.32
Alphabet City,14354,"233,040.60",16.24,"36,444.46"
Arrochar/Fort Wadsworth,10,649.45,64.94,94.65
Astoria,6286,"106,404.71",16.93,"17,334.88"
Astoria Park,74,"2,339.00",31.61,391.72


In [266]:
area_zone_df['average_tip'] = area_zone_df['tip_amount'] / area_zone_df['Number_of_trips']
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip
PULocationZone,,,,,
Allerton/Pelham Gardens,14,375.14,26.80,87.32,6.24
Alphabet City,14354,"233,040.60",16.24,"36,444.46",2.54
Arrochar/Fort Wadsworth,10,649.45,64.94,94.65,9.46
Astoria,6286,"106,404.71",16.93,"17,334.88",2.76
Astoria Park,74,"2,339.00",31.61,391.72,5.29


In [271]:
area_zone_df['Number_of_trips_weight'] = area_zone_df['Number_of_trips'] / float(area_zone_df['Number_of_trips'].sum()) * 100
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight
PULocationZone,,,,,,
Allerton/Pelham Gardens,14,375.14,26.80,87.32,6.24,0.00
Alphabet City,14354,"233,040.60",16.24,"36,444.46",2.54,0.23
Arrochar/Fort Wadsworth,10,649.45,64.94,94.65,9.46,0.00
Astoria,6286,"106,404.71",16.93,"17,334.88",2.76,0.10
Astoria Park,74,"2,339.00",31.61,391.72,5.29,0.00


In [277]:
area_zone_df['average_tip_X_Number_of_trips_weight'] = area_zone_df['average_tip'] * area_zone_df['Number_of_trips_weight']
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight
PULocationZone,,,,,,,
Allerton/Pelham Gardens,14,375.14,26.80,87.32,6.24,0.00,0.00
Alphabet City,14354,"233,040.60",16.24,"36,444.46",2.54,0.23,0.59
Arrochar/Fort Wadsworth,10,649.45,64.94,94.65,9.46,0.00,0.00
Astoria,6286,"106,404.71",16.93,"17,334.88",2.76,0.10,0.28
Astoria Park,74,"2,339.00",31.61,391.72,5.29,0.00,0.01


In [278]:
area_zone_df['average_taxi_amount_X_Number_of_trips_weight'] = area_zone_df['average_taxi_amount'] * area_zone_df['Number_of_trips_weight']
area_zone_df.head()

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight,average_taxi_amount_X_Number_of_trips_weight
PULocationZone,,,,,,,,
Allerton/Pelham Gardens,14,375.14,26.80,87.32,6.24,0.00,0.00,0.01
Alphabet City,14354,"233,040.60",16.24,"36,444.46",2.54,0.23,0.59,3.76
Arrochar/Fort Wadsworth,10,649.45,64.94,94.65,9.46,0.00,0.00,0.01
Astoria,6286,"106,404.71",16.93,"17,334.88",2.76,0.10,0.28,1.72
Astoria Park,74,"2,339.00",31.61,391.72,5.29,0.00,0.01,0.04


In [280]:
area_zone_df.sort_values(by=['average_taxi_amount_X_Number_of_trips_weight'],ascending=False)

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight,average_taxi_amount_X_Number_of_trips_weight
PULocationZone,,,,,,,,
LaGuardia Airport,201275,"9,510,894.88",47.25,"1,562,527.59",7.76,3.25,25.23,153.55
JFK Airport,117402,"7,314,794.32",62.31,"1,150,573.78",9.80,1.90,18.58,118.10
Midtown Center,230705,"3,791,671.85",16.44,"621,565.15",2.69,3.72,10.04,61.22
Midtown East,222951,"3,605,899.62",16.17,"589,262.33",2.64,3.60,9.51,58.22
Times Sq/Theatre District,178216,"3,309,709.76",18.57,"548,193.67",3.08,2.88,8.85,53.43
Murray Hill,213915,"3,309,591.67",15.47,"533,105.57",2.49,3.45,8.61,53.43
Penn Station/Madison Sq West,199753,"3,231,963.88",16.18,"536,926.06",2.69,3.22,8.67,52.18
Union Sq,217285,"3,230,810.70",14.87,"519,430.40",2.39,3.51,8.39,52.16
Upper East Side South,238038,"3,114,666.56",13.08,"500,870.12",2.10,3.84,8.09,50.29


In [281]:
area_zone_df.sort_values(by=['average_tip_X_Number_of_trips_weight'],ascending=False)

,Number_of_trips,total_amount,average_taxi_amount,tip_amount,average_tip,Number_of_trips_weight,average_tip_X_Number_of_trips_weight,average_taxi_amount_X_Number_of_trips_weight
PULocationZone,,,,,,,,
LaGuardia Airport,201275,"9,510,894.88",47.25,"1,562,527.59",7.76,3.25,25.23,153.55
JFK Airport,117402,"7,314,794.32",62.31,"1,150,573.78",9.80,1.90,18.58,118.10
Midtown Center,230705,"3,791,671.85",16.44,"621,565.15",2.69,3.72,10.04,61.22
Midtown East,222951,"3,605,899.62",16.17,"589,262.33",2.64,3.60,9.51,58.22
Times Sq/Theatre District,178216,"3,309,709.76",18.57,"548,193.67",3.08,2.88,8.85,53.43
Penn Station/Madison Sq West,199753,"3,231,963.88",16.18,"536,926.06",2.69,3.22,8.67,52.18
Murray Hill,213915,"3,309,591.67",15.47,"533,105.57",2.49,3.45,8.61,53.43
Union Sq,217285,"3,230,810.70",14.87,"519,430.40",2.39,3.51,8.39,52.16
Upper East Side South,238038,"3,114,666.56",13.08,"500,870.12",2.10,3.84,8.09,50.29
